# Dashboard Liderazgo

In [ ]:
import os

from_drive = True  # same flag you use everywhere

if os.environ.get("ATLAS_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
        branch_to_pull = 'dev'

        os.chdir('/content')

        if not os.path.isdir('Atlas'):
            !git clone {git_url}

        %cd Atlas
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        !pip install -r PipelinesConsumo/src/requirements.txt
        %cd PipelinesConsumo

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    os.environ["ATLAS_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done, assuming orchestrator ran it.")

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns,
                       remove_accents)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from PipelinesConsumo.src.processedAtlas import ProcessedAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive)
from utils.utils import get_dates_dataframe
from src.constants import (atlas_raw_output_folder_id,
                           atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           data_source_folder_id,
                           raw_output_ids,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           id_torre_de_control,
                           )


warnings.filterwarnings('ignore')



In [ ]:
consumo_sheets_ids_dict

## Load

In [ ]:
ventas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcVentas'])
cancelaciones = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPublicacionesCanceladas'])
publicaciones = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPublicaciones'])
pedidos = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPedidos'])
visitas_unicas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcVisitasUnicas'])
adobe_vendedor_total = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcAdobeFunnelVendedorTotal'])
adobe_comprador_total = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcAdobeFunnelCompradorTotal'])
clientes = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcClientes'])
business_case_df = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcBusinessCase26'])

### load insumos apis

In [ ]:
try:
  folder_id_catalogos = "1TE24Yl4lQ6ZxHJSw_ZDzb6W4XpB-21FL"
  id_catalogos_status = list_file_ids_for_drive_folder(drive,folder_id_catalogos)['CatalogoSolicitudesAprobacion']
  cat_tarea_actual = (read_from_google_sheets(gc,id_catalogos_status,sheetname='CatTareaActual')
                      .assign(tareaactual = lambda x: x.nb_tarea_actual.apply(remove_accents).str.strip().str.upper(),
                              status_automarket = lambda x: x.status_am.apply(remove_accents).str.strip().str.upper())
                      .drop(columns=['nb_tarea_actual','status_am'])
                      )
  cat_decision_sistema = (read_from_google_sheets(gc,id_catalogos_status,sheetname='CatDecisionSistema')
                        .assign(decisionsistema = lambda x: x.decisionsistema.apply(remove_accents).str.strip().str.upper(),
                                status_riesgos = lambda x: x.riesgos.apply(remove_accents).str.strip().str.upper())
                        .drop(columns=['riesgos'])
                          )

  # edas, gabo, torre de control
  edas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcEdas'])
  bauto = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcConsolidadoBautoLastStatus'])
  torreRaw = read_from_google_sheets(gc,id_torre_de_control,sheetname='Asignación compradores')
  torreRaw.columns
  clientes = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcClientes'])
  # clientes
  subset_columns = ['id_lead','origen_automarket','id_comprador','id_de_ultimo_pedido','folio_bauto',
                    'espacio_automarket','asesor_de_ventas','fecha_de_asignacion','total_apartados',
                    'estatus_de_lead','lead_contactado_(visualiza_cc)',
                    'fecha_de_cierre_del_lead','motivos_de_cancelacion',
                    'correo_recibido_en_buzon_contingencia','documentacion_completa_contingencia',
                    ]
  rename_dict = {'lead_contactado_(visualiza_cc)':'lead_contactado'}

  torre = (torreRaw
  .pipe(process_columns)
  [subset_columns]
  .assign(folio = lambda x: pd.to_numeric(x['folio_bauto'],errors='coerce').astype('Int64'),
          id_comprador = lambda x: pd.to_numeric(x['id_comprador'],errors='coerce').astype('Int64'),
          fecha_de_asignacion = lambda x: pd.to_datetime(x['fecha_de_asignacion'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
          #  fecha_de_cierre_del_lead = lambda x: pd.to_datetime(x['fecha_de_cierre_del_lead'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
          )
  .rename(columns=rename_dict)
  .sort_values(by='fecha_de_asignacion',ascending=False)
  )

  folder_id_folios_rod = '1OW4yxE7h8BCcn0mqhCfk05B4_27Ghvfd'
  files_rod = list_file_ids_for_drive_folder(drive,folder_id_folios_rod)
  files = list(files_rod.keys())
  files = [x for x in files if '.xlsx' in x]
  latest_rod_id = files_rod.get(files[0]) ## cambiar este siempre
  print('Fecha archivo folios rod:')
  print(files[0])
  latest_rod_id
  from_drive_to_local(drive,latest_rod_id,'rod_latest.xlsx')
  rod = (pd.read_excel('rod_latest.xlsx')
        .rename(columns={'Name':'intento',
                        'MX_ATN_Id_Simulacion__c':'n_simulacion',
                        'MX_ATN_creditId__c':'folio',
                        'MX_ATN_Account__r.Name':'name',
                        'MX_ATN_Account__r.MX_ATN_CommerceId__c':'id_am',
                        'MX_ATN_Account__r.MX_ATN_PrimaryContact__r.Email':'email',
                        'MX_ATN_Account__r.MX_ATN_PrimaryContact__r.MobilePhone':'phone',
                        'MX_ATN_Status__c':'status'})
        .assign(CreatedDate = lambda x: pd.to_datetime(x.CreatedDate, format="%d/%m/%Y, %H:%M"),
                phone = lambda x: x.phone.astype('Int64').astype(str))
        )
  flag_insumos_api=1
except Exception as e:
  print('Error en load de insumos para funnel api. No se incluirá en la master table.')
  print(e)
  flag_insumos_api=0

## Process

In [ ]:
datedf = get_dates_dataframe(start='2024-03-01').assign(date = lambda x: x.date.dt.strftime('%Y-%m-%d'))

In [ ]:
# clientes  
clientes_by_date = (clientes
 .rename(columns={'customer_since':'date'})
 .groupby('date')['id_am'].nunique()
 .to_frame()
 .rename(columns={'id_am':'usuarios_registrados'})
 )

In [ ]:
# visitas unicas
visitas_date = (visitas_unicas
 .groupby(['date']).sum()
 )

In [ ]:
# funnel vendedor
sum_cols_vendedor = adobe_vendedor_total.filter(like='fv_').columns
rename_dict_vendedor = {'fv_app_click_start':'fv_0_app_click_start',
                        'fv_app_page_visit':'fv_1_marca_modelo',
                        'fv_app_step_2':'fv_2_version',
                        'fv_app_step_3':'fv_3_tenemos_tu_version',
                        'fv_app_step_4':'fv_4_precio',
                        'fv_app_step_6':'fv_5_datos_complementarios',
                        'fv_app_step_8':'fv_6_espacio_automarket',
                        'fv_app_step_9':'fv_7_fotografias',
                        'fv_app_step_10':'fv_8_exito_publicacion'
                        }
funnel_vendedor_date = (adobe_vendedor_total
    .groupby(['date'])[sum_cols_vendedor].sum()
    .rename(columns=rename_dict_vendedor)
)

In [ ]:
# funnel comprador
sum_cols_comprador = adobe_comprador_total.filter(like='fc_').columns
rename_dict_comprador = {'fc_app_click_start':'fc_0_app_click_start',
                        'fc_app_page_visit':'fc_1_onboarding',
                        'fc_app_completed':'fc_2_apartado_exitoso',
                        }
funnel_comprador_date = (adobe_comprador_total
    .groupby(['date'])[sum_cols_comprador].sum()
    .rename(columns=rename_dict_comprador)
)

In [ ]:
# cancelaciones
cancelaciones_date = (cancelaciones
 .rename(columns={'cancelled_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones_canceladas')
 .to_frame()
 )

In [ ]:
# publicaciones
publicaciones_date = (publicaciones
 [lambda x: x.published_at.notna()]
  .rename(columns={'published_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones')
 .to_frame() 
 )
showroom_map = {'Reforma 510':'torre',
                'Metrópoli Patriotismo':'patriotismo',
                'Samara Satélite':'samara'}
publicaciones_espacio_date = (publicaciones
 [lambda x: x.published_at.notna()]
 .assign(year_month=lambda x:pd.to_datetime( x['published_at']).dt.strftime('%Y-%m'),
         showroom = lambda x:x.showroom.map(showroom_map))
  .rename(columns={'published_at':'date'})
 .groupby(['date','showroom'])['sku'].nunique()
 .unstack('showroom')
 .fillna(0)
 )
publicaciones_espacio_date.columns = [f'publicaciones_{showroom}' for showroom in publicaciones_espacio_date.columns]
publicaciones_activas_date = (publicaciones
 [lambda x: x.flag_publicacion_activa==1]
.rename(columns={'published_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones_activas')
 .to_frame()
 )

In [ ]:
# pedidos
pedidos_date = (pedidos
 .rename(columns={'fecha_de_creacion':'date'})
 .groupby(['date','multiapartado'])
 ['sf_order_id'].nunique()
 .unstack('multiapartado')
 .rename(columns={0:'apartados_unicos',
               1:'multiapartados'})
 .fillna(0)
 .assign(apartados = lambda x: x.sum(axis=1))
 )

In [ ]:
# ventas
ventas_pago_date = (ventas
 .rename(columns={'fecha_de_entrega':'date'})
 .groupby(['date','tipo_de_venta'])
 .size()
 .rename({0:'ventas'})
 .unstack('tipo_de_venta')
 .fillna(0)
 )
ventas_pago_date.columns = [f'ventas_{tipo}' for tipo in ventas_pago_date.columns]
ventas_pago_date = ventas_pago_date.assign(ventas = lambda x: x.sum(axis=1))
ventas_espacio_date = (ventas
 .rename(columns={'fecha_de_entrega':'date'})
 .groupby(['date','espacio_am'])
 .size()
 .rename({0:'ventas'})
 .unstack('espacio_am')
 .fillna(0)
 )
ventas_espacio_date.columns = [f'ventas_{showroom}' for showroom in ventas_espacio_date.columns]

In [ ]:
# business case 
bc_diario = (business_case_df
    .assign(days_in_month = lambda x: pd.to_datetime(x.mes).dt.days_in_month,
            year_month = lambda x: pd.to_datetime(x.mes).dt.strftime('%Y-%m'))
    
    )
bc_diario = (bc_diario
    .assign(**{c:bc_diario[c]/bc_diario['days_in_month'] for c in bc_diario.columns 
               if c not in ['mes','days_in_month','year_month']})
    .drop(columns=['mes','days_in_month'])
    )

### process apis

In [ ]:
if flag_insumos_api==1:
        bauto_mod_api = (bauto
                .assign(flag_eda = lambda x: np.where(x.folio.isin(edas.folio.unique()),1,0),
                        flag_tc = lambda x: np.where(x.folio.isin(torre.folio.unique()),1,0),
                        origen_real= lambda x: np.where(x.folio.isin(edas.folio.unique()),'EDA',x.origen),
                        decisionsistema = lambda x: x.decisionsistema.apply(remove_accents).str.strip().str.upper(),
                        tareaactual = lambda x: x.tareaactual.apply(remove_accents).str.strip().str.upper(),
                        telefono = lambda x: x.telefono.astype('Int64')
                        )
                [lambda x: x.origen_real=='API']
                .merge(cat_tarea_actual,on='tareaactual',how='left')
                .merge(cat_decision_sistema,on='decisionsistema',how='left')
                .drop_duplicates()
                .sort_values(by='fecha_creacion',ascending=False)
                )


        dummies_status_df = pd.get_dummies(bauto_mod_api.status_automarket)*1
        dummies_status_df.columns = [f'status_automarket_{x}' for x in dummies_status_df.columns]
        bauto_mod_api = pd.concat([bauto_mod_api,dummies_status_df],axis=1)


        bauto_by_date=(bauto_mod_api
        .groupby('fecha_creacion',as_index=False)
        .agg(bauto_folios_api=('folio','nunique'),
        bauto_folios_en_proceso = ('status_automarket_PROCESO DESEMBOLSO','sum'),
        bauto_folios_desembolso = ('status_automarket_DESEMBOLSADO','sum'),
        bauto_folios_rechazado = ('status_automarket_RECHAZADO','sum'),
        bauto_folios_inicio_solicitud = ('status_automarket_INICIO SOLICITUD','sum'),
        bauto_folios_revision_riesgo = ('status_automarket_REVISION RIESGOS','sum'),
        
        )
        .rename(columns={'fecha_creacion':'date'})

        )

In [ ]:
if flag_insumos_api==1:
     rod_by_date = (rod
                    .assign(date = lambda x:pd.to_datetime(x.CreatedDate).dt.strftime('%Y-%m-%d'),
                         id_am_con_folio = lambda x: np.where(x.folio.notna(),x.id_am,np.nan))
                    .groupby('date',as_index=False)
                    .agg(fapi_intentos_solicitud=('intento','nunique'),
                         fapi_usuarios_con_intento=('id_am','nunique'),
                         fapi_folios=('folio','nunique'),
                         fapi_usuarios_con_folio=('id_am_con_folio','nunique'),
                         )
                    )

In [ ]:
master = (datedf
          .merge(visitas_date,on='date',how='left')
          .merge(funnel_vendedor_date,on='date',how='left')
          .merge(funnel_comprador_date,on='date',how='left')
          .merge(publicaciones_date,on='date',how='left')
          .merge(publicaciones_activas_date,on='date',how='left')
          .merge(publicaciones_espacio_date,on='date',how='left')
          .merge(cancelaciones_date,on='date',how='left')
          .merge(pedidos_date,on='date',how='left')
          .merge(ventas_espacio_date,on='date',how='left')
          .merge(ventas_pago_date,on='date',how='left')
          .merge(clientes_by_date,on='date',how='left')
          .fillna(0)
          .assign(year = lambda x:pd.to_datetime(x.date).dt.year,
                  month = lambda x:pd.to_datetime(x.date).dt.month,
                  year_month=lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%m'),
                  year_week = lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%U'),
                  day_of_week = lambda x:pd.to_datetime( x['date']).dt.strftime('%A'),
                  monday_of_week=lambda x: (pd.to_datetime(x.date) - 
                                            pd.to_timedelta(pd.to_datetime(x.date).dt.dayofweek, unit="d")).dt.strftime("%Y-%m-%d"),
                  )
          .merge(bc_diario,on='year_month',how='left')
          
          )
if flag_insumos_api==1:
        print('Se incluirá funnel api en master table...')
        master = (master
                .merge(rod_by_date,on='date',how='left')
                .merge(bauto_by_date,on='date',how='left')
                )

## write

In [ ]:
output_master_table_id = '1v8EAje4iFtzSHVoNjQusmjGSjhhnvzaH6vVDlYh0b4U'
update_sheets_in_drive_folder(gc,output_master_table_id,'Hoja 1',master)

## free memory

In [ ]:
vars_to_del = ['ventas','ventas_espacio_date','ventas_pago_date','business_case_df',
               'publicaciones','publicaciones_activas_date','publicaciones_date','publicaciones_espacio_date',
               'pedidos','pedidos_date',
               'cancelaciones_date','cancelaciones',
               'funnel_comprador_date','funnel_vendedor_date',
               'visitas_date',
               'clientes',
               'adobe_comprador_total','adobe_vendedor_total','bc_diario',
               'rod_by_date',"bauto","bauto_by_date","bauto_mod_api",
               "rod","edas","torreRaw","torre","dummies_status_df",
               "cat_tarea_actual","cat_decision_sistema"]
for v in vars_to_del:
    try:
        del globals()[v]
    except Exception as e:
        print(f"could not delete var {v}: {e}")

In [ ]:
import gc as gcol
gcol.collect()